In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/my-cv-e/Menna_Osama_AI.pdf


In [2]:
!pip install langchain langchain-community pypdf transformers torch fastapi uvicorn pyngrok nest-asyncio sentence-transformers gradio

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 76.0 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.41.5
    Uninstalling pydantic_core-2.41.5:
      Successfully uninstalled pydantic_core-2.41.5
  Attempting uninstall: packaging
    Found existing installation: packaging 26.0rc2
    Uninstalling packaging-26.0rc2:
      Successfully uninstalled packaging-26.0rc2
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.12.5
    Un

In [3]:
import os
import torch
import json
import pandas as pd
import gradio as gr
from pyngrok import ngrok
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

model_name = "mistralai/Mistral-Nemo-Instruct-2407"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

def generate_text(prompt, max_length=1500):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
2026-02-03 18:53:08.989873: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770144789.421163      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770144789.558978      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770144790.613785      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770144790.613823      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770144790.613826      55

...

model-00001-of-00005.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [4]:
skill = ResponseSchema(name="skill", description="The name of the required technical or soft skill.")
exists = ResponseSchema(name="exists", description="Does this skill exist in the CV? Answer with (Yes/No).")
improvement = ResponseSchema(name="improvement", description="Professional advice on how to develop this skill.")

response_schemas = [skill, exists, improvement]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

template = """
You are a professional HR Recruitment Expert. 
Analyze the Resume against the Job Description.

Step 1: Extract important skills from Job Description.
Step 2: Check if each skill is in the Resume.
Step 3: Provide recommendations for missing skills.

cv Text:
{cv_text}

Job Description:
{job_desc}

{format_instructions}

Return ONLY the JSON list.
"""

final_template = template.replace("{format_instructions}", format_instructions.replace("{", "{{").replace("}", "}}"))

def extract_json_robust(text):
    try:
        start = text.find("[")
        end = text.rfind("]") + 1
        if start != -1 and end != 0:
            json_str = text[start:end]
            data = json.loads(json_str)
            return pd.DataFrame(data)
        return "No JSON found in response."
    except Exception as e:
        return f"Error parsing JSON: {str(e)}"

In [5]:
def graduate_project_interface(cv_file, job_description):
    try:
        loader = PyPDFLoader(cv_file.name)
        cv_text = "\n".join([doc.page_content for doc in loader.load()])
        
        prompt_input = final_template.format(cv_text=cv_text, job_desc=job_description)
        
        raw_response = generate_text(prompt_input)
        
        df = extract_json_robust(raw_response)
        
        if isinstance(df, pd.DataFrame):
            df_clean = df[~df['skill'].str.contains('string|skill name', case=False, na=False)].copy()
            return df_clean
        else:
            return pd.DataFrame([{"Error": "Could not parse AI response. Try again."}])
    except Exception as e:
        return pd.DataFrame([{"Error": str(e)}])

demo = gr.Interface(
    fn=graduate_project_interface,
    inputs=[
        gr.File(label="Upload your CV (PDF)"), 
        gr.Textbox(label="Job Description", lines=5, placeholder="Paste requirements here...")
    ],
    outputs=gr.Dataframe(label="Skill Match Analysis"),
    title="AI Resume Analyzer - HR Expert System",
    description="Analyze how well a CV matches a job description using Mistral-Nemo LLM."
)

In [6]:
NGROK_TOKEN = "YOUR_NGROK_TOKEN" 
ngrok.set_auth_token(NGROK_TOKEN)

ngrok.kill()

public_url = ngrok.connect(7860).public_url
print(f"Click here to open the UI: {public_url}")

demo.launch(share=True)

Click here to open the UI: https://tegularly-exhaustless-dottie.ngrok-free.dev                      
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://0cbe3ad0cade7103de.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Created dataset file at: .gradio/flagged/dataset1.csv
